In [ ]:
def main():
    file_path = "/Users/jaffar/Desktop/example1.SAC"
    st = obspy.read(file_path)
    amplitude = st[0].data
    sampling_rate = st[0].stats.sampling_rate
    start_time = st[0].stats.starttime.datetime
    timestamp = [start_time + timedelta(seconds=i / sampling_rate) for i in range(len(amplitude))]
    
    window_duration = 10  # seconds
    window_size = int(sampling_rate * window_duration)  # Samples per window
    fractal_dimensions = []
    time_labels = []  # Store datetime objects for the x-axis

    output_path = "/Users/jaffar/Desktop/output.txt"
    with open(output_path, 'w') as output_file: 
        for step in range(1, len(amplitude) // window_size):
            start_index = (step - 1) * window_size
            end_index = min(start_index + window_size, len(amplitude))
            segment = amplitude[start_index:end_index]

            if len(segment) >= 2:
                _, gamma_tau_list = calculate_variogram(segment, sampling_rate)
                fractal_dimension = calculate_fractal_dimension(gamma_tau_list, sampling_rate)
            else:
                fractal_dimension = None

            # Only append if fractal dimension is valid
            if fractal_dimension is not None:
                fractal_dimensions.append(fractal_dimension)
                step_start_time = timestamp[start_index + window_size // 2]  # Midpoint time for the segment
                time_labels.append(step_start_time)
                output_file.write(f"{step},{step_start_time}, {fractal_dimension}\n")  # Save to file

    # Plotting the waveform, and fractal dimensions over time
    fig, axs = plt.subplots(2, 1, figsize=(15, 12), constrained_layout=True)

    # Plot waveform
    axs[0].plot(timestamp, amplitude, color='black')
    axs[0].set_title('Waveform (BHZ Component)')
    axs[0].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)  # Hide x-axis labels
    axs[0].set_ylabel('Amplitude')

    # Plot fractal dimensions over time
    axs[1].plot(time_labels, fractal_dimensions, label='Fractal Dimension', marker='o')
    axs[1].set_ylim(1, 2)  # Set y-limits to [1, 2] for fractal dimension
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Fractal Dimension')
    axs[1].set_title('Fractal Dimension over Time')
    axs[1].xaxis.set_major_formatter(plt.FixedFormatter([t.strftime("%H:%M") for t in time_labels[::60]]))
    axs[1].set_xticks(time_labels[::60])

    plt.show()

if __name__ == "__main__":
    main()
